# data preparation

In [97]:
# read data
import pandas as pd
data = pd.read_csv('freelancer_profiles.csv')
print(data)

                    Location           Industry  Years of Experience     Pay  \
0          São Paulo, Brazil    Web Development                    9   63.45   
1            Shanghai, China       UI/UX Design                    4   51.10   
2                 London, UK      Video Editing                    1   82.02   
3    Cape Town, South Africa        Photography                    7  116.45   
4            Toronto, Canada      Video Editing                    7  118.98   
..                       ...                ...                  ...     ...   
195          Berlin, Germany      Video Editing                   11   57.74   
196        Sydney, Australia        Photography                    8  106.95   
197        Sydney, Australia  Digital Marketing                   18   98.43   
198             Tokyo, Japan     Graphic Design                   10   61.15   
199          Shanghai, China     Graphic Design                    1  114.16   

     Rating (out of 5)  
0             

In [2]:
!pip install transformers torch

In [3]:
!pip install -U sentence-transformers
!pip install torch transformers

import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 69.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 72.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [98]:
data

,Location,Industry,Years of Experience,Pay,Rating (out of 5)
0,"São Paulo, Brazil",Web Development,9,63.45,3.4
1,"Shanghai, China",UI/UX Design,4,51.10,3.7
2,"London, UK",Video Editing,1,82.02,3.1
3,"Cape Town, South Africa",Photography,7,116.45,3.0
4,"Toronto, Canada",Video Editing,7,118.98,4.1
...,...,...,...,...,...
195,"Berlin, Germany",Video Editing,11,57.74,4.7
196,"Sydney, Australia",Photography,8,106.95,4.5
197,"Sydney, Australia",Digital Marketing,18,98.43,3.6
198,"Tokyo, Japan",Graphic Design,10,61.15,3.5


In [99]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder_location = LabelEncoder()
label_encoder_industry = LabelEncoder()

data['Location'] = label_encoder_location.fit_transform(data['Location'])
data['Industry'] = label_encoder_industry.fit_transform(data['Industry'])

data

,Location,Industry,Years of Experience,Pay,Rating (out of 5)
0,7,9,9,63.45,3.4
1,5,7,4,51.10,3.7
2,2,8,1,82.02,3.1
3,1,5,7,116.45,3.0
4,9,8,7,118.98,4.1
...,...,...,...,...,...
195,0,8,11,57.74,4.7
196,6,5,8,106.95,4.5
197,6,3,18,98.43,3.6
198,8,4,10,61.15,3.5


In [100]:
# test/train split where X is every column but pay and target variable y is pay
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 

X=data.drop(columns=['Pay'])
y=data['Pay'] 
  
# using the train test split function 
X_train, X_test, y_train, y_test = train_test_split(X,y , 
                                   random_state=104,  
                                   test_size=0.25,  
                                   shuffle=True) 

In [101]:
# print(X_train)
#print(y)
print(y_train)

73      79.31
30      96.81
23     109.17
155    113.17
157     81.33
        ...  
127     82.55
166     25.59
142     81.77
193     30.58
69      87.20
Name: Pay, Length: 150, dtype: float64


In [102]:
# normalize x train and test
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform it
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Optional: Convert the scaled data back to DataFrame for easier interpretation
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Normalized Training Data:")
print(X_train_scaled.head())

print("\nNormalized Testing Data:")
print(X_test_scaled.head())

Normalized Training Data:
   Location  Industry  Years of Experience  Rating (out of 5)
0 -0.499032 -0.637749             1.389386          -0.584734
1  0.203830 -1.325967             1.217432           1.054708
2 -0.147601  1.426906             0.701571          -0.584734
3  1.609553  1.082797            -0.502105           1.054708
4 -0.499032  0.394578            -1.361873           0.398931

Normalized Testing Data:
   Location  Industry  Years of Experience  Rating (out of 5)
0 -0.147601 -0.981858            -0.674058           1.218652
1 -0.499032  0.394578            -0.330151          -0.912623
2 -1.201893  0.394578            -0.158197           0.398931
3  0.555261 -1.670076            -1.705781           1.218652
4  1.609553  0.050469            -1.361873           1.054708


# move the data into an s3 bucket

In [104]:
# # keep structure including target in x_train
# X_train_scaled.dropna(subset=['target'], inplace=True)
# print(X_train_scaled)
# save files locally
X_train_scaled.to_csv('train_data.csv', header=False, index=False)
X_test_scaled.to_csv('test_data.csv', header=False, index=False)

In [105]:
import boto3
bucket_name = 'sagemaker-build-and-deploy-market'

# Bucket and file setup
train_key = 'train/train_data.csv'
test_key = 'test/test_data.csv'

# Upload files to S3
s3 = boto3.resource('s3')
s3.Bucket(bucket_name).upload_file('train_data.csv', train_key)
s3.Bucket(bucket_name).upload_file('test_data.csv', test_key)

# create model

In [106]:
import sagemaker
from sagemaker import LinearLearner
from sagemaker import get_execution_role

# SageMaker session
sagemaker_session = sagemaker.Session()

# Automatically get the IAM role for SageMaker execution
role = get_execution_role()

# Linear Learner Estimator
linear_estimator = LinearLearner(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    predictor_type='regressor',  # 'regressor' for linear regression
    output_path=f's3://{bucket_name}/output',
    sagemaker_session=sagemaker_session,
)

# train model

In [108]:
from sagemaker.inputs import TrainingInput
# Set up the training data input (this will still use the S3 path)
# train_s3_path = f's3://{bucket_name}/train/train_data.csv'
# train_input = TrainingInput(s3_data='s3://{bucket_name}/train/train_data.csv', content_type='csv')

# linear_estimator.fit(train_input)
from sagemaker.amazon.amazon_estimator import RecordSet

# Assume `data_location` is the S3 URI for your data
record_set = RecordSet(s3_data='s3://{bucket_name}/train/train_data.csv', num_records=1000, feature_dim=10)

# Fit the model
linear_estimator.fit(record_set)


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2024-11-17-10-45-58-885
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: Could not download manifest file with S3 URL "s3://%7Bbucket_name%7D/train/train_data.csv". Please ensure that the bucket exists in the selected region (us-west-2), that the manifest file exists at that S3 URL, and that the role "arn:aws:iam::592768992043:role/service-role/AmazonSageMaker-ExecutionRole-20241116T154026" has "s3:GetObject" permissions on the manifest file. Error message from S3: The specified bucket is not valid.

import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Step 1: Set up environment
bucket_name = 'sagemaker-market-analysis2'
prefix = 'linear-regression-example'
role = get_execution_role()
region = boto3.Session().region_name

# Step 2: Load and preprocess the data
df = pd.read_csv('freelancer_profiles.csv')

# Separate target and features
X = df.drop('Pay', axis=1)
y = df['Pay']

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Concatenate target and features back together for uploading
train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

# Save as CSV without header
train_data.to_csv('train.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)

# Step 3: Upload to S3
s3 = boto3.client('s3')
s3.upload_file('train.csv', bucket_name, f'{prefix}/train/train.csv')
s3.upload_file('test.csv', bucket_name, f'{prefix}/test/test.csv')

# Step 4: Configure the LinearLearner estimator
linear_estimator = Estimator(
    image_uri=sagemaker.image_uris.retrieve('linear-learner', region),
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    sagemaker_session=sagemaker.Session()
)

linear_estimator.set_hyperparameters(
    predictor_type='regressor',  # Regression task
    mini_batch_size=100
)

# Step 5: Fit the model
train_data_s3_path = f's3://{bucket_name}/{prefix}/train/train.csv'
linear_estimator.fit({'train': train_data_s3_path}, job_name='linear-regression-training-job')

# Step 6: Deploy the model
predictor = linear_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)

# Step 7: Use the model for prediction
predictions = predictor.predict(X_test)

# Step 8: Clean up
predictor.delete_endpoint()
